In [1]:
import pandas as pd
from config import OUTPUT_DIR
from os.path import join as pjoin

In [2]:
csv_file = pjoin(OUTPUT_DIR,"n2c2.csv")
df = pd.read_csv(csv_file)
df

,Unnamed: 0,sample_id,raw,annotated
0,0,108809,Admission Date: [**2112-10-6**] ...,T1\tDrug 19078 19085\tAspirin\nT2\tStrength 19...
1,1,115143,Admission Date: [**2146-4-3**] D...,T1\tDrug 5569 5576;5577 5583\tInsulin Lispro\n...
2,2,159079,Admission Date: [**2110-8-7**] D...,T1\tStrength 10013 10017\t20mg\nR1\tStrength-D...
3,3,102027,Admission Date: [**2119-9-16**] ...,T1\tDrug 1014 1017\tFFP\nT2\tDosage 1171 1178\...
4,4,102296,Admission Date: [**2172-9-28**] ...,T1\tDrug 112 122\tQuinolones\nT4\tDrug 1135 11...
...,...,...,...,...
298,298,130440,Admission Date: [**2185-7-6**] D...,T1\tDrug 3838 3844\tflagyl\nT2\tDrug 6019 6032...
299,299,118510,Admission Date: [**2183-1-9**] D...,T1\tDrug 11649 11656\tBicitra\nT3\tDrug 14147 ...
300,300,174150,Admission Date: [**2186-6-14**] ...,T1\tDrug 1674 1682\tCefepime\nT2\tStrength 168...
301,301,107047,Admission Date: [**2134-10-9**] ...,T1\tStrength 14094 14099\t20 mg\nT2\tForm 1411...


In [3]:
from io import StringIO
index = 0
sub_df = pd.read_csv(StringIO(df['annotated'].iloc[index]),sep="\t",header=None,names=["type","name","value"])

In [4]:
sub_df

,type,name,value
0,T1,Drug 19078 19085,Aspirin
1,T2,Strength 19086 19092,325 mg
2,T3,Route 19093 19095,PO
3,T4,Drug 19465 19478;19479 19482,Ciprofloxacin HCl
4,T5,Strength 19483 19489,500 mg
...,...,...,...
168,R69,Reason-Drug Arg1:T102 Arg2:T59,NaN
169,R70,Reason-Drug Arg1:T103 Arg2:T16,NaN
170,R71,ADE-Drug Arg1:T94 Arg2:T8,NaN
171,T93,Route 12791 12793,IV


In [5]:
from collections import defaultdict
all_drugs = defaultdict(lambda : defaultdict(lambda : []))

for i, row in sub_df.iterrows():
    if row['name'].startswith("Reason-Drug"):
        reason_id, drug_id = [val.split(":")[1] for val in row["name"].split()[1:]]
        all_drugs[drug_id]["reason"].append(reason_id)
    elif row['name'].startswith("ADE-Drug"):
        ade_id, drug_id = [val.split(":")[1] for val in row["name"].split()[1:]]
        all_drugs[drug_id]["ade"].append(ade_id)

In [6]:
def find_term(term,df=sub_df):
    return df[df['type'] == term].iloc[0]['value']

In [7]:
from collections import namedtuple
SpecialCase = namedtuple("SpecialCase","drug reason ade")
values = [SpecialCase(find_term(k), find_term(v['reason'][0]), find_term(v['ade'][0])) for k, v in all_drugs.items() if 'reason' in v and 'ade' in v]

In [8]:
values

[SpecialCase(drug='Ativan', reason='agitated', ade='delerious')]

In [9]:
print(df['raw'].iloc[0])

Admission Date:  [**2112-10-6**]              Discharge Date:   [**2112-10-14**]

Date of Birth:  [**2055-4-7**]             Sex:   F

Service: SURGERY

Allergies:
Ativan

Attending:[**First Name3 (LF) 2777**]
Chief Complaint:
Abdominal Pain

Major Surgical or Invasive Procedure:
[**2112-10-10**] Thoracic Endovascular Aortic Aneurysm Repair


History of Present Illness:
HPI: 57F heavy smoker with recent admission [**2112-10-1**] - [**2112-10-3**]
with
chest and abdominal pain and odynophagia who was found to have
circumferential mural thrombus in the supra-renal aorta on cross
sectional imaging. At that time the aortic pathology was though
to be chronic. Ultimately, her pain resolved with the initiation
of a PPI and GI cocktail, and was discharged home after a 3 day
hospital stay. On [**10-6**] she again presented to an OSH with
abdominal pain where a CT abdomen was done which showed an
aortic
dissection. She was emergently transferred to [**Hospital1 18**] where a CTA
of the torso con

In [10]:
sub_df

,type,name,value
0,T1,Drug 19078 19085,Aspirin
1,T2,Strength 19086 19092,325 mg
2,T3,Route 19093 19095,PO
3,T4,Drug 19465 19478;19479 19482,Ciprofloxacin HCl
4,T5,Strength 19483 19489,500 mg
...,...,...,...
168,R69,Reason-Drug Arg1:T102 Arg2:T59,NaN
169,R70,Reason-Drug Arg1:T103 Arg2:T16,NaN
170,R71,ADE-Drug Arg1:T94 Arg2:T8,NaN
171,T93,Route 12791 12793,IV


In [11]:
print(df.iloc[0]['annotated'])

T1	Drug 19078 19085	Aspirin
T2	Strength 19086 19092	325 mg
T3	Route 19093 19095	PO
T4	Drug 19465 19478;19479 19482	Ciprofloxacin HCl
T5	Strength 19483 19489	500 mg
T6	Frequency 19493 19497	Q12H
T7	Route 19490 19492	PO
T8	Drug 17932 17938	ativan
T9	Drug 19345 19355;19355 19374	butalbital -acetaminophen-caff
T10	Drug 19612 19622;19623 19631	Metoprolol Tartrate
T11	Strength 19632 19639	12.5 mg
T12	Frequency 19643 19646	BID
T13	Route 19640 19642	PO
T14	Drug 18974 18984	Omeprazole
T15	Strength 18985 18990	40 mg
T16	Drug 17824 17830	Ativan
T17	Drug 19105 19116	DiCYCLOmine
T18	Strength 19117 19122	10 mg
T19	Frequency 19126 19129	QID
T20	Route 19123 19125	PO
T21	Drug 20771 20778	Aspirin
T22	Drug 164 170	Ativan
T23	Strength 19375 19380;19380 19393	50 mg -325 mg-40 mg
T24	Drug 19247 19257	Omeprazole
T25	Strength 19258 19263	40 mg
T26	Route 19264 19266	PO
T27	Drug 19276 19295;19295 19305	Acetaminophen-Caff- Butalbital
T28	Form 19317 19320	TAB
T29	Frequency 19324 19327;19327 19331	Q6H :PRN
T30	Rou

In [12]:
'''Finds all entity in all texts'''
import pandas as pd
import spacy
import scispacy
from scispacy.linking import EntityLinker

# CHUNK_SIZE = 1000
# df = pd.read_pickle("notes.pickle")
pipeline = spacy.load("en_core_sci_sm")
pipeline.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})


/home/james/programs/miniconda3/envs/mimic-env/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/home/james/programs/miniconda3/envs/mimic-env/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [14]:
from util import load_pickle

In [15]:
obj = load_pickle("result.pickle")

In [17]:
ent = pipeline("delirium")

/home/james/programs/miniconda3/envs/mimic-env/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/james/programs/miniconda3/envs/mimic-env/lib/python3.7/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [23]:
ent.ents[0].label

17590775310403619837

In [24]:
ent.ents[0].kb_id

0

In [ ]:
ent.ents[0].

In [22]:
ent.ents[0].label_

'ENTITY'

In [ ]:
ent.ents[0].

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("abhibisht89/spanbert-large-cased-finetuned-ade_corpus_v2")
model = AutoModelForTokenClassification.from_pretrained("abhibisht89/spanbert-large-cased-finetuned-ade_corpus_v2")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
from transformers import pipeline
nlp = pipeline("ner",model=model,tokenizer=tokenizer)